In [7]:
import numpy as np
import matplotlib.pyplot as plt
import json # mark: add this line
import pandas as pd

In [8]:
# data source: https://huggingface.co/datasets/dair-ai/emotion

In [9]:
from datasets import load_dataset
dataset = load_dataset("dair-ai/emotion")

No config specified, defaulting to: emotion/split
Found cached dataset emotion (file://C:/Users/mark7/.cache/huggingface/datasets/dair-ai___emotion/split/1.0.0/cca5efe2dfeb58c1d098e0f9eeb200e9927d889b5a03c67097275dfb5fe463bd)


NotImplementedError: Loading a dataset cached in a LocalFileSystem is not supported.

In [ ]:
dataset